In [4]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice
import datetime as datetime
import glob
from matplotlib import pyplot
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
import warnings

%matplotlib inline

plt.style.use('default')
sns.set(style="whitegrid")
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 5)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from catboost import CatBoostRegressor

In [5]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler
# the model
from sklearn.ensemble import RandomForestClassifier
# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline
# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [8]:
train_labels = pd.read_csv('train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
train_values = pd.read_csv('train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [9]:
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [10]:
train = train_values.merge(train_labels, on='building_id')
train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,2
669485,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
602512,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,3


In [11]:
train = train.drop('geo_level_1_id', axis=1)
train = train.drop('geo_level_2_id', axis=1)
train = train.drop('geo_level_3_id', axis=1)
train = train.drop('land_surface_condition', axis=1)
#train = train.drop('count_floors_pre_eq', axis=1)

In [12]:
#edad promedio segun cantidad de pisos
train['edad_promedio'] = train.groupby('count_floors_pre_eq')['age'].transform('mean')

In [13]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,0,0,3,25.76
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,0,0,2,25.76
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,0,0,3,25.76
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,0,0,2,25.76
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,0,0,3,30.16


In [14]:
#cantidad de tipos de techo segun edad
train['tipo_techo'] = train.groupby('age')['roof_type'].transform('count')
#train = train.drop('roof_type', axis=1)

In [15]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,0,3,25.76,18028
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,0,2,25.76,38896
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,0,3,25.76,38896
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,0,2,25.76,38896
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,0,3,30.16,18028


In [16]:
#cantidad de familias por edificacion
train['flias_afectadas'] = train.groupby('building_id')['count_families'].transform('count')

In [17]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,3,25.76,18028,1
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,2,25.76,38896,1
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,3,25.76,38896,1
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,2,25.76,38896,1
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,3,30.16,18028,1


In [18]:
train = train.drop('roof_type', axis=1)
train = train.drop('other_floor_type', axis=1)
train = train.drop('position', axis=1)
#train = train.drop('other_floor_type', axis=1)
train = train.drop('plan_configuration', axis=1)
train = train.drop('legal_ownership_status', axis=1)
train = train.drop('ground_floor_type', axis=1)
train = train.drop('foundation_type', axis=1)

In [19]:
train_value = train['damage_grade']
train_data = train.iloc[:, train.columns != 'damage_grade']

In [20]:
train_data

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,25.76,18028,1
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.76,38896,1
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.76,38896,1
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.76,38896,1
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,30.16,18028,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,1,55,6,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,17.82,2033,1
669485,2,0,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.76,26041,1
602512,3,55,6,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,30.16,2033,1


test_labels = pd.read_csv('test_train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
test_values = pd.read_csv('test_train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

test = test_values.merge(test_labels, on='building_id')

test = test.drop('geo_level_1_id', axis=1)
test = test.drop('geo_level_2_id', axis=1)
test = test.drop('geo_level_3_id', axis=1)
test = test.drop('land_surface_condition', axis=1)

#edad promedio segun cantidad de pisos
test['edad_promedio'] = test.groupby('count_floors_pre_eq')['age'].transform('mean')

#cantidad de tipos de techo segun edad
test['tipo_techo'] = test.groupby('age')['roof_type'].transform('count')

#cantidad de familias por edificacion
test['cantidad_flias'] = test.groupby('building_id')['count_families'].transform('count')

test = test.drop('roof_type', axis=1)
test = test.drop('other_floor_type', axis=1)
test = test.drop('position', axis=1)
#test = test.drop('other_floor_type', axis=1)
test = test.drop('plan_configuration', axis=1)
test = test.drop('legal_ownership_status', axis=1)
test = test.drop('ground_floor_type', axis=1)
test = test.drop('foundation_type', axis=1)


#test = test.drop_duplicates('damage_grade')
IDs = test[['damage_grade']].copy()
#test = test.drop('damage_grade', axis=1)

test.head()

test_value = test['damage_grade']
test_data = test.iloc[:, test.columns != 'damage_grade']

test.head()

test_data.head()

In [21]:
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=False)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(train_data, train_value, train_size=0.7, random_state=1234)

In [23]:
train_data

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,25.76,18028,1
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.76,38896,1
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.76,38896,1
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.76,38896,1
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,30.16,18028,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,1,55,6,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,17.82,2033,1
669485,2,0,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.76,26041,1
602512,3,55,6,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,30.16,2033,1


In [24]:
categorical_features_indices = np.where(train_data.dtypes == np.object)[0]

In [30]:
%%capture
modelCatBoost=CatBoostRegressor(iterations=2000, depth=5, learning_rate=0.1)
modelCatBoost.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation),plot=True)

0:	learn: 0.6019916	test: 0.6018653	best: 0.6018653 (0)	total: 26.5ms	remaining: 53s
1:	learn: 0.5938568	test: 0.5936255	best: 0.5936255 (1)	total: 51.7ms	remaining: 51.7s
2:	learn: 0.5871399	test: 0.5868259	best: 0.5868259 (2)	total: 84.6ms	remaining: 56.3s
3:	learn: 0.5818086	test: 0.5814235	best: 0.5814235 (3)	total: 117ms	remaining: 58.6s
4:	learn: 0.5772206	test: 0.5767838	best: 0.5767838 (4)	total: 151ms	remaining: 1m
5:	learn: 0.5734441	test: 0.5729242	best: 0.5729242 (5)	total: 185ms	remaining: 1m 1s
6:	learn: 0.5701662	test: 0.5696136	best: 0.5696136 (6)	total: 218ms	remaining: 1m 1s
7:	learn: 0.5674704	test: 0.5668333	best: 0.5668333 (7)	total: 250ms	remaining: 1m 2s
8:	learn: 0.5647925	test: 0.5641295	best: 0.5641295 (8)	total: 273ms	remaining: 1m
9:	learn: 0.5624201	test: 0.5617279	best: 0.5617279 (9)	total: 301ms	remaining: 59.9s
10:	learn: 0.5608025	test: 0.5601008	best: 0.5601008 (10)	total: 327ms	remaining: 59.2s
11:	learn: 0.5593954	test: 0.5586678	best: 0.5586678 (11)

95:	learn: 0.5451425	test: 0.5444909	best: 0.5444909 (95)	total: 2.5s	remaining: 49.7s
96:	learn: 0.5451176	test: 0.5444687	best: 0.5444687 (96)	total: 2.53s	remaining: 49.7s
97:	learn: 0.5450940	test: 0.5444499	best: 0.5444499 (97)	total: 2.55s	remaining: 49.6s
98:	learn: 0.5450792	test: 0.5444461	best: 0.5444461 (98)	total: 2.58s	remaining: 49.5s
99:	learn: 0.5450409	test: 0.5444131	best: 0.5444131 (99)	total: 2.6s	remaining: 49.4s
100:	learn: 0.5450161	test: 0.5443996	best: 0.5443996 (100)	total: 2.62s	remaining: 49.3s
101:	learn: 0.5449716	test: 0.5443618	best: 0.5443618 (101)	total: 2.65s	remaining: 49.3s
102:	learn: 0.5449494	test: 0.5443472	best: 0.5443472 (102)	total: 2.67s	remaining: 49.3s
103:	learn: 0.5449052	test: 0.5443118	best: 0.5443118 (103)	total: 2.7s	remaining: 49.3s
104:	learn: 0.5448672	test: 0.5442785	best: 0.5442785 (104)	total: 2.73s	remaining: 49.2s
105:	learn: 0.5448268	test: 0.5442387	best: 0.5442387 (105)	total: 2.75s	remaining: 49.2s
106:	learn: 0.5447814	t

190:	learn: 0.5429835	test: 0.5428363	best: 0.5428363 (190)	total: 4.79s	remaining: 45.4s
191:	learn: 0.5429551	test: 0.5428150	best: 0.5428150 (191)	total: 4.82s	remaining: 45.4s
192:	learn: 0.5429275	test: 0.5427990	best: 0.5427990 (192)	total: 4.85s	remaining: 45.4s
193:	learn: 0.5429183	test: 0.5427958	best: 0.5427958 (193)	total: 4.88s	remaining: 45.4s
194:	learn: 0.5428996	test: 0.5427804	best: 0.5427804 (194)	total: 4.91s	remaining: 45.4s
195:	learn: 0.5428665	test: 0.5427482	best: 0.5427482 (195)	total: 4.93s	remaining: 45.4s
196:	learn: 0.5428416	test: 0.5427433	best: 0.5427433 (196)	total: 4.95s	remaining: 45.3s
197:	learn: 0.5428194	test: 0.5427277	best: 0.5427277 (197)	total: 4.98s	remaining: 45.3s
198:	learn: 0.5428001	test: 0.5427226	best: 0.5427226 (198)	total: 5.01s	remaining: 45.3s
199:	learn: 0.5427654	test: 0.5426984	best: 0.5426984 (199)	total: 5.03s	remaining: 45.3s
200:	learn: 0.5427425	test: 0.5426778	best: 0.5426778 (200)	total: 5.05s	remaining: 45.2s
201:	learn

285:	learn: 0.5416356	test: 0.5420230	best: 0.5420230 (285)	total: 7.26s	remaining: 43.5s
286:	learn: 0.5416188	test: 0.5420177	best: 0.5420177 (286)	total: 7.28s	remaining: 43.5s
287:	learn: 0.5416076	test: 0.5420198	best: 0.5420177 (286)	total: 7.32s	remaining: 43.5s
288:	learn: 0.5415934	test: 0.5420027	best: 0.5420027 (288)	total: 7.34s	remaining: 43.5s
289:	learn: 0.5415846	test: 0.5419973	best: 0.5419973 (289)	total: 7.37s	remaining: 43.5s
290:	learn: 0.5415652	test: 0.5419916	best: 0.5419916 (290)	total: 7.41s	remaining: 43.5s
291:	learn: 0.5415544	test: 0.5419845	best: 0.5419845 (291)	total: 7.43s	remaining: 43.5s
292:	learn: 0.5415299	test: 0.5419661	best: 0.5419661 (292)	total: 7.46s	remaining: 43.5s
293:	learn: 0.5415236	test: 0.5419653	best: 0.5419653 (293)	total: 7.49s	remaining: 43.5s
294:	learn: 0.5415158	test: 0.5419615	best: 0.5419615 (294)	total: 7.51s	remaining: 43.4s
295:	learn: 0.5415059	test: 0.5419540	best: 0.5419540 (295)	total: 7.54s	remaining: 43.4s
296:	learn

384:	learn: 0.5407719	test: 0.5416552	best: 0.5416552 (384)	total: 9.74s	remaining: 40.9s
385:	learn: 0.5407610	test: 0.5416439	best: 0.5416439 (385)	total: 9.76s	remaining: 40.8s
386:	learn: 0.5407579	test: 0.5416437	best: 0.5416437 (386)	total: 9.78s	remaining: 40.8s
387:	learn: 0.5407546	test: 0.5416415	best: 0.5416415 (387)	total: 9.8s	remaining: 40.7s
388:	learn: 0.5407484	test: 0.5416404	best: 0.5416404 (388)	total: 9.83s	remaining: 40.7s
389:	learn: 0.5407267	test: 0.5416377	best: 0.5416377 (389)	total: 9.85s	remaining: 40.7s
390:	learn: 0.5407168	test: 0.5416260	best: 0.5416260 (390)	total: 9.88s	remaining: 40.7s
391:	learn: 0.5406932	test: 0.5416252	best: 0.5416252 (391)	total: 9.91s	remaining: 40.6s
392:	learn: 0.5406811	test: 0.5416110	best: 0.5416110 (392)	total: 9.94s	remaining: 40.7s
393:	learn: 0.5406753	test: 0.5416097	best: 0.5416097 (393)	total: 9.99s	remaining: 40.7s
394:	learn: 0.5406702	test: 0.5416057	best: 0.5416057 (394)	total: 10s	remaining: 40.7s
395:	learn: 0

...

1610:	learn: 0.5355283	test: 0.5405702	best: 0.5405679 (1490)	total: 40.6s	remaining: 9.81s
1611:	learn: 0.5355259	test: 0.5405718	best: 0.5405679 (1490)	total: 40.7s	remaining: 9.79s
1612:	learn: 0.5355234	test: 0.5405708	best: 0.5405679 (1490)	total: 40.7s	remaining: 9.77s
1613:	learn: 0.5355180	test: 0.5405655	best: 0.5405655 (1613)	total: 40.7s	remaining: 9.74s
1614:	learn: 0.5355081	test: 0.5405658	best: 0.5405655 (1613)	total: 40.7s	remaining: 9.71s
1615:	learn: 0.5355065	test: 0.5405656	best: 0.5405655 (1613)	total: 40.8s	remaining: 9.69s
1616:	learn: 0.5355048	test: 0.5405648	best: 0.5405648 (1616)	total: 40.8s	remaining: 9.66s
1617:	learn: 0.5355031	test: 0.5405660	best: 0.5405648 (1616)	total: 40.8s	remaining: 9.64s
1618:	learn: 0.5355002	test: 0.5405662	best: 0.5405648 (1616)	total: 40.8s	remaining: 9.61s
1619:	learn: 0.5354996	test: 0.5405674	best: 0.5405648 (1616)	total: 40.9s	remaining: 9.59s
1620:	learn: 0.5354969	test: 0.5405662	best: 0.5405648 (1616)	total: 40.9s	remai

1707:	learn: 0.5352844	test: 0.5405648	best: 0.5405601 (1686)	total: 43.2s	remaining: 7.38s
1708:	learn: 0.5352806	test: 0.5405643	best: 0.5405601 (1686)	total: 43.2s	remaining: 7.35s
1709:	learn: 0.5352798	test: 0.5405654	best: 0.5405601 (1686)	total: 43.2s	remaining: 7.33s
1710:	learn: 0.5352786	test: 0.5405662	best: 0.5405601 (1686)	total: 43.2s	remaining: 7.3s
1711:	learn: 0.5352762	test: 0.5405662	best: 0.5405601 (1686)	total: 43.3s	remaining: 7.28s
1712:	learn: 0.5352739	test: 0.5405655	best: 0.5405601 (1686)	total: 43.3s	remaining: 7.25s
1713:	learn: 0.5352709	test: 0.5405643	best: 0.5405601 (1686)	total: 43.3s	remaining: 7.22s
1714:	learn: 0.5352695	test: 0.5405658	best: 0.5405601 (1686)	total: 43.3s	remaining: 7.2s
1715:	learn: 0.5352615	test: 0.5405625	best: 0.5405601 (1686)	total: 43.4s	remaining: 7.19s
1716:	learn: 0.5352594	test: 0.5405619	best: 0.5405601 (1686)	total: 43.4s	remaining: 7.16s
1717:	learn: 0.5352578	test: 0.5405630	best: 0.5405601 (1686)	total: 43.5s	remaini

1804:	learn: 0.5350572	test: 0.5405640	best: 0.5405485 (1723)	total: 45.9s	remaining: 4.95s
1805:	learn: 0.5350568	test: 0.5405634	best: 0.5405485 (1723)	total: 45.9s	remaining: 4.93s
1806:	learn: 0.5350553	test: 0.5405630	best: 0.5405485 (1723)	total: 45.9s	remaining: 4.9s
1807:	learn: 0.5350540	test: 0.5405644	best: 0.5405485 (1723)	total: 45.9s	remaining: 4.88s
1808:	learn: 0.5350531	test: 0.5405649	best: 0.5405485 (1723)	total: 46s	remaining: 4.85s
1809:	learn: 0.5350523	test: 0.5405653	best: 0.5405485 (1723)	total: 46s	remaining: 4.83s
1810:	learn: 0.5350501	test: 0.5405666	best: 0.5405485 (1723)	total: 46s	remaining: 4.8s
1811:	learn: 0.5350483	test: 0.5405680	best: 0.5405485 (1723)	total: 46s	remaining: 4.78s
1812:	learn: 0.5350467	test: 0.5405688	best: 0.5405485 (1723)	total: 46.1s	remaining: 4.75s
1813:	learn: 0.5350420	test: 0.5405699	best: 0.5405485 (1723)	total: 46.1s	remaining: 4.72s
1814:	learn: 0.5350410	test: 0.5405689	best: 0.5405485 (1723)	total: 46.1s	remaining: 4.7s

1900:	learn: 0.5348485	test: 0.5405646	best: 0.5405485 (1723)	total: 48.4s	remaining: 2.52s
1901:	learn: 0.5348469	test: 0.5405634	best: 0.5405485 (1723)	total: 48.4s	remaining: 2.49s
1902:	learn: 0.5348431	test: 0.5405654	best: 0.5405485 (1723)	total: 48.5s	remaining: 2.47s
1903:	learn: 0.5348418	test: 0.5405666	best: 0.5405485 (1723)	total: 48.5s	remaining: 2.44s
1904:	learn: 0.5348395	test: 0.5405639	best: 0.5405485 (1723)	total: 48.5s	remaining: 2.42s
1905:	learn: 0.5348385	test: 0.5405648	best: 0.5405485 (1723)	total: 48.5s	remaining: 2.39s
1906:	learn: 0.5348369	test: 0.5405648	best: 0.5405485 (1723)	total: 48.5s	remaining: 2.37s
1907:	learn: 0.5348365	test: 0.5405654	best: 0.5405485 (1723)	total: 48.6s	remaining: 2.34s
1908:	learn: 0.5348347	test: 0.5405640	best: 0.5405485 (1723)	total: 48.6s	remaining: 2.31s
1909:	learn: 0.5348329	test: 0.5405635	best: 0.5405485 (1723)	total: 48.6s	remaining: 2.29s
1910:	learn: 0.5348306	test: 0.5405631	best: 0.5405485 (1723)	total: 48.6s	remai

1995:	learn: 0.5346706	test: 0.5405441	best: 0.5405406 (1959)	total: 50.8s	remaining: 102ms
1996:	learn: 0.5346649	test: 0.5405429	best: 0.5405406 (1959)	total: 50.8s	remaining: 76.3ms
1997:	learn: 0.5346639	test: 0.5405431	best: 0.5405406 (1959)	total: 50.8s	remaining: 50.9ms
1998:	learn: 0.5346633	test: 0.5405440	best: 0.5405406 (1959)	total: 50.8s	remaining: 25.4ms
1999:	learn: 0.5346618	test: 0.5405442	best: 0.5405406 (1959)	total: 50.9s	remaining: 0us

bestTest = 0.5405405502
bestIteration = 1959

Shrink model to first 1960 iterations.


In [35]:
test = pd.read_csv('test_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [36]:
test.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0


In [37]:
test = test.drop('geo_level_1_id', axis=1)
test = test.drop('geo_level_2_id', axis=1)
test = test.drop('geo_level_3_id', axis=1)
test = test.drop('land_surface_condition', axis=1)
#test = test.drop('count_floors_pre_eq', axis=1)

In [38]:
#edad promedio segun cantidad de pisos
test['edad_promedio'] = test.groupby('count_floors_pre_eq')['age'].transform('mean')

In [39]:
#cantidad de tipos de techo segun edad
test['tipo_techo'] = test.groupby('age')['roof_type'].transform('count')
#train = train.drop('roof_type', axis=1)

In [40]:
#cantidad de familias por edificacion
test['flias_afectadas'] = test.groupby('building_id')['count_families'].transform('count')

In [41]:
test = test.drop('roof_type', axis=1)
test = test.drop('other_floor_type', axis=1)
test = test.drop('position', axis=1)
#test = test.drop('other_floor_type', axis=1)
test = test.drop('plan_configuration', axis=1)
test = test.drop('legal_ownership_status', axis=1)
test = test.drop('ground_floor_type', axis=1)
test = test.drop('foundation_type', axis=1)

In [42]:
preds = modelCatBoost.predict(test)

In [56]:
test.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
300051,3,20,7,6,0,1,0,0,0,0,...,0,0,0,0,0,0,0,29.80,10610,1
99355,2,25,13,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.72,8220,1
890251,2,5,4,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.72,11348,1
745817,1,0,19,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,18.11,8684,1
421793,3,15,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,29.80,12064,1


In [58]:
ids = test.reset_index()['building_id']

In [62]:
ids

0         300051
1          99355
2         890251
3         745817
4         421793
          ...   
86863     310028
86864     663567
86865    1049160
86866     442785
86867     501372
Name: building_id, Length: 86868, dtype: int64

In [60]:
preds

array([2.45596252, 2.3086627 , 2.36117026, ..., 2.34203663, 2.22871824,
       1.73231758])

In [64]:
damages = pd.Series(preds).rename("damage_grade")

In [76]:
submit = pd.merge(ids, damages, left_index=True,right_index=True).set_index('building_id')

submit['damage_grade'] = submit['damage_grade'].round(0)
submit = submit.astype({'damage_grade':'int32'})
submit.to_csv('submit_001.csv')

In [71]:
submit['damage_grade'].round(0)

building_id
300051                    2.00
99355                     2.00
890251                    2.00
745817                    2.00
421793                    2.00
                  ...         
310028                    2.00
663567                    3.00
1049160                   2.00
442785                    2.00
501372                    2.00
Name: damage_grade, Length: 86868, dtype: float64

In [149]:
preds = modelCatBoost.predict(X_train)
logloss = log_loss(y_train.replace(3, 1), preds)     #tener solo 2 opciones - REVISAR
print("logloss: %f" % (logloss))

logloss: 14.854712


In [ ]:
pd.to_numeric(preds.round(), downcast='signed')

In [132]:
y_train.replace(3, 1)

building_id
323246     2
666127     1
990198     2
934274     2
490051     2
          ..
847186     1
685157     2
741415     1
894354     2
1018556    1
Name: damage_grade, Length: 93553, dtype: uint8

In [ ]:
type(preds)

In [136]:
%%capture
modelCatBoost.fit(train_data,train_value,cat_features=categorical_features_indices)
preds = modelCatBoost.predict(test)
entregable_CatBoost = IDs[['damage_grade']].copy()
entregable_CatBoost['Target'] = preds

0:	learn: 0.6074042	total: 72ms	remaining: 17.9s
1:	learn: 0.6035335	total: 111ms	remaining: 13.7s
2:	learn: 0.6000186	total: 148ms	remaining: 12.2s
3:	learn: 0.5968143	total: 182ms	remaining: 11.2s
4:	learn: 0.5939748	total: 217ms	remaining: 10.6s
5:	learn: 0.5912467	total: 249ms	remaining: 10.1s
6:	learn: 0.5888345	total: 337ms	remaining: 11.7s
7:	learn: 0.5865535	total: 381ms	remaining: 11.5s
8:	learn: 0.5844426	total: 425ms	remaining: 11.4s
9:	learn: 0.5823927	total: 468ms	remaining: 11.2s
10:	learn: 0.5806813	total: 509ms	remaining: 11.1s
11:	learn: 0.5789347	total: 579ms	remaining: 11.5s
12:	learn: 0.5772925	total: 620ms	remaining: 11.3s
13:	learn: 0.5758940	total: 651ms	remaining: 11s
14:	learn: 0.5746288	total: 688ms	remaining: 10.8s
15:	learn: 0.5733878	total: 723ms	remaining: 10.6s
16:	learn: 0.5722886	total: 774ms	remaining: 10.6s
17:	learn: 0.5711159	total: 931ms	remaining: 12s
18:	learn: 0.5700555	total: 984ms	remaining: 12s
19:	learn: 0.5690232	total: 1.02s	remaining: 11.

164:	learn: 0.5494346	total: 9.26s	remaining: 4.77s
165:	learn: 0.5494081	total: 9.32s	remaining: 4.72s
166:	learn: 0.5493732	total: 9.36s	remaining: 4.65s
167:	learn: 0.5493314	total: 9.4s	remaining: 4.59s
168:	learn: 0.5493149	total: 9.45s	remaining: 4.53s
169:	learn: 0.5492985	total: 9.61s	remaining: 4.52s
170:	learn: 0.5492536	total: 9.71s	remaining: 4.48s
171:	learn: 0.5492363	total: 9.75s	remaining: 4.42s
172:	learn: 0.5492256	total: 9.79s	remaining: 4.36s
173:	learn: 0.5492082	total: 9.94s	remaining: 4.34s
174:	learn: 0.5491763	total: 10s	remaining: 4.3s
175:	learn: 0.5491622	total: 10.1s	remaining: 4.24s
176:	learn: 0.5491492	total: 10.1s	remaining: 4.17s
177:	learn: 0.5491430	total: 10.2s	remaining: 4.13s
178:	learn: 0.5491099	total: 10.3s	remaining: 4.09s
179:	learn: 0.5490920	total: 10.4s	remaining: 4.04s
180:	learn: 0.5490833	total: 10.4s	remaining: 3.97s
181:	learn: 0.5490536	total: 10.5s	remaining: 3.91s
182:	learn: 0.5490134	total: 10.5s	remaining: 3.85s
183:	learn: 0.54

In [137]:
entregable_CatBoost.to_csv('CatBoost.csv', index=False)

In [138]:
modeloCB = CatBoostRegressor()
LogLoss = make_scorer(log_loss, greater_is_better = False)
categorical_features_indices = np.where(train_data.dtypes == np.object)[0]

In [141]:
parameters = {'iterations':[100, 250, 500] , 'depth':[2, 3, 5], 'learning_rate':[0.005, 0.05, 0.5]}
gscv = GridSearchCV(modeloCB, parameters, cv=5, scoring=LogLoss)

In [142]:
train_data

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,9299,1
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,30.18,9299,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,2,60,11,6,0,0,0,0,0,1,...,0,0,0,0,0,0,0,25.82,1809,1
349181,2,0,8,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,13307,1
1009561,1,5,15,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17.64,17361,1


In [151]:
train_values = train_values.iloc[:133648]

In [152]:
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,10,1382,11935,2,60,11,6,t,u,x,...,0,0,0,0,0,0,0,0,0,0
349181,13,1154,1044,2,0,8,4,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1009561,26,1001,11008,1,5,15,3,t,i,x,...,0,0,0,0,0,0,0,0,0,0


In [153]:
%%capture
gscv.fit(train_data, train_values, cat_features = categorical_features_indices)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already speci

Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already speci

CatBoostError: catboost/private/libs/target/data_providers.cpp:533: Currently only multi-regression and survival objectives work with multidimensional target

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

In [154]:
print(gscv.best_params_)

{'depth': 2, 'iterations': 100, 'learning_rate': 0.005}


In [157]:
%%capture
modeloCB = CatBoostRegressor(depth = 5, iterations = 100, learning_rate = 0.05)
modeloCB.fit(train_data, train_value, cat_features = categorical_features_indices)
result = modeloCB.predict(test)

entregable_CB = IDs[['damage_grade']].copy() #revisar
entregable_CB['Target'] = result #revisar

0:	learn: 0.6066649	total: 15.6ms	remaining: 1.54s
1:	learn: 0.6022365	total: 32.4ms	remaining: 1.59s
2:	learn: 0.5980752	total: 48.6ms	remaining: 1.57s
3:	learn: 0.5942718	total: 64.2ms	remaining: 1.54s
4:	learn: 0.5908039	total: 80.4ms	remaining: 1.53s
5:	learn: 0.5876481	total: 96ms	remaining: 1.5s
6:	learn: 0.5847817	total: 114ms	remaining: 1.51s
7:	learn: 0.5822441	total: 131ms	remaining: 1.5s
8:	learn: 0.5798911	total: 146ms	remaining: 1.48s
9:	learn: 0.5777652	total: 162ms	remaining: 1.46s
10:	learn: 0.5758224	total: 178ms	remaining: 1.44s
11:	learn: 0.5738011	total: 194ms	remaining: 1.42s
12:	learn: 0.5721527	total: 210ms	remaining: 1.41s
13:	learn: 0.5706226	total: 247ms	remaining: 1.52s
14:	learn: 0.5689638	total: 263ms	remaining: 1.49s
15:	learn: 0.5676843	total: 279ms	remaining: 1.47s
16:	learn: 0.5664625	total: 295ms	remaining: 1.44s
17:	learn: 0.5653921	total: 311ms	remaining: 1.41s
18:	learn: 0.5643840	total: 326ms	remaining: 1.39s
19:	learn: 0.5632746	total: 341ms	remai

In [159]:
entregable_CB.to_csv('CB_Final.csv', index=False)